挂载Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


文件是压缩文件时运行：

In [ ]:
!cp /content/drive/My\ Drive/Art.zip ./
!mkdir Art
!unzip -d ./Art Art.zip

文件已经解压时运行：

In [ ]:
!cp -r /content/drive/My\ Drive/Art ./

In [ ]:
from __future__ import print_function, division #导入python未来支持的打印语句(print_function)和精确除法(division)

import torch
import torch.nn as nn #集成了一些常见的网络层，如卷积等
import torch.optim as optim #集成了许多常用的优化算法，如SGD，Adam等
from torch.optim import lr_scheduler #lr_scheduler提供了多种基于epoch训练次数进行学习率调整的方法
from PIL import Image #PIL（Python Imaging Library）是Python常用的图像处理库
import pandas as pd #pandas是基于NumPy的一种工具，该工具是为了解决数据分析任务而创建的。
import os #操作系统接口模块
from torchvision import models, transforms #torchvision包是服务于pytorch深度学习框架的,用来生成图片,视频数据集,和一些流行的模型类和预训练模型.
                        #torchvision.models : Definitions for popular model architectures, such as AlexNet, VGG, and ResNet and pre-trained models.
                        #torchvision.transforms : Common image transformations such as random crop, rotations etc.
import time


device = torch.device("cuda:0" if torch.cuda.is_available() else "CPU" )
print(device)
# model = models.resnet101()
# print(model)

cuda:0


In [ ]:

! mkdir /content/drive/My\ Drive/checkpoint

mkdir: cannot create directory ‘/content/drive/My Drive/checkpoint’: File exists


In [ ]:
import pandas as pd

root_dir = "./Art"

#  类别分布是不平衡的，但是可以先试试这样的结果怎么样

def get_img_map(dir):
  file = pd.read_csv(dir) #读取CSV（逗号分割）文件
  img_map = dict({}) #dict()函数用于创建一个字典，dict({})：创建空字典
  img_cnt = dict({})

  for i in range(len(file)):
    img_map[i]=file.iloc[i][1]
    if file.iloc[i][1] in img_cnt.keys(): #keys()函数以列表返回一个字典所有的键
      img_cnt[file.iloc[i][1]] += 1
    else:
      img_cnt[file.iloc[i][1]] = 1

  return img_map,img_cnt



In [ ]:
class ImageDataset(torch.utils.data.Dataset): #torch.utils.data.Dataset是一个抽象类，用来读取数据集

    #找出键值对中最大的value值
    def get_max_cnt(self):
      self.max_cnt = 0
      for k,v in self.img_cnt.items(): #items()函数以列表返回可遍历的(键,值)元组数组。
        self.max_cnt = max(v, self.max_cnt)

    def __init__(self, img_label, img_cnt, images_folder, transforms=None):
        self.img_list = [i for i in range(len(img_label))]
        self.img_label = img_label
        self.img_cnt = img_cnt
        self.images_folder = images_folder
        self.get_max_cnt()
        self.transforms = transforms
        for i in range(len(img_label)):
          for j in range(1,self.max_cnt//img_cnt[img_label[i]]):
            self.img_list.append(i)
        self.num_examples = len(self.img_list)
        print("{}, {}".format(self.max_cnt, self.num_examples))

    def __len__(self):
        return self.num_examples

    def __getitem__(self, index):
        label = torch.tensor(self.img_label[self.img_list[index]], dtype=torch.long)
        feature = Image.open(os.path.join(self.images_folder, str(self.img_list[index])+".jpg"))
        if feature.mode != 'RGB':
          feature = feature.convert("RGB")
        if self.transforms:
            feature = self.transforms(feature)
        sample = (feature, label)
        return sample


In [ ]:
img_map,img_cnt = get_img_map(os.path.join(root_dir, "train.csv"))

num_class = len(img_map)


transform = {
    'train':transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ]),
}


train_set = ImageDataset(img_map, img_cnt, os.path.join(root_dir, "train"), transform['train'])
train_loader = torch.utils.data.DataLoader(train_set, batch_size=64, shuffle=True, num_workers=6)

img_num = train_set.__len__()

790, 35337


In [ ]:
def initialize_model(model_name, num_class, finetuning=False, pretrained=True):

    if model_name == 'resnet18':
        model = models.resnet18(pretrained=pretrained)
    elif model_name == 'resnet34':
        model = models.resnet34(pretrained=pretrained)
    elif model_name == 'resnet50':
        model = models.resnet50(pretrained=pretrained)
    elif model_name == 'resnet101':
        model = models.resnet101(pretrained=pretrained)
    elif model_name == 'resnet152':
        model = models.resnet152(pretrained=pretrained)        
    else:
        raise ValueError("No such Model %s" % model_name)

    if finetuning:
      pass
    else:
      for param in model.parameters():
        param.requires_grad = False    

    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, num_class)
    model = model.to(device)
    return model


In [ ]:
def train_model(model, criterion, optimizer,pre_epoch, num_epochs):
    since = time.time()
    time_once = since
    for epoch in range(pre_epoch, num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)  

        running_loss = 0.0
        running_corrects = 0

        # Iterate over data.
        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss/img_num
        epoch_acc = running_corrects.double()/img_num

        print('Loss: {:.4f} Acc: {:.4f}'.format(
           epoch_loss, epoch_acc))
        print("Training Time per Epoch {}". format(time.time()-time_once)) 
        time_once = time.time()
        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    return model




In [ ]:
model = initialize_model('resnet152', num_class, False)
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)    
criterion = nn.CrossEntropyLoss()
pre_epoch = 0

check_name = r'/content/drive/My Drive/checkpoint/152_state-best.tar'

if "152_state-best.tar" in os.listdir(r'/content/drive/My Drive/checkpoint'):  
    print('loading previous state............')
    checkpoint = torch.load(check_name)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

model = train_model(model, criterion, optimizer, pre_epoch, 15)
check_name = r'/content/drive/My Drive/checkpoint/152_state-best.tar'

torch.save({
    'model_state_dict':model.state_dict(),
    'optimizer_state_dict':optimizer.state_dict()
},check_name)

torch.save({
    'model_state_dict':model.state_dict(),
    'optimizer_state_dict':optimizer.state_dict()
},check_name)



Downloading: "https://download.pytorch.org/models/resnet152-b121ed2d.pth" to /root/.cache/torch/hub/checkpoints/resnet152-b121ed2d.pth



loading previous state............
Epoch 0/14
----------
Loss: 0.2446 Acc: 0.9258
Training Time per Epoch 231.83077192306519

Epoch 1/14
----------
Loss: 0.2494 Acc: 0.9242
Training Time per Epoch 230.52148938179016

Epoch 2/14
----------
Loss: 0.2485 Acc: 0.9243
Training Time per Epoch 231.40835642814636

Epoch 3/14
----------
Loss: 0.2498 Acc: 0.9236
Training Time per Epoch 232.61332273483276

Epoch 4/14
----------
Loss: 0.2462 Acc: 0.9241
Training Time per Epoch 233.18987917900085

Epoch 5/14
----------
Loss: 0.2420 Acc: 0.9266
Training Time per Epoch 232.38924765586853

Epoch 6/14
----------
Loss: 0.2496 Acc: 0.9242
Training Time per Epoch 231.673166513443

Epoch 7/14
----------
Loss: 0.2409 Acc: 0.9264
Training Time per Epoch 231.53832459449768

Epoch 8/14
----------
Loss: 0.2486 Acc: 0.9240
Training Time per Epoch 231.29383373260498

Epoch 9/14
----------


In [ ]:
test_folder = os.path.join(root_dir, "test")
test_img = os.listdir(test_folder)

ans = [0]*len(test_img)
ansf = open(r'/content/drive/My Drive/submission5.txt','w')
model.eval()
for img in test_img:
  image = Image.open(os.path.join(test_folder, img))
  if image.mode != 'RGB':
    image = image.convert("RGB")
  image = transform['val'](image)
  image = image.unsqueeze(0)
  image = image.to(device)
  index = int(os.path.splitext(img)[0])
  print(index)
  output = model(image)
  _,preds = torch.max(output.data,1)
  ans[index]=preds.item()
for i in range(len(ans)):
  print(i, ans[i], file=ansf, sep=',')
ansf.close()


173
196
19
474
304
379
49
77
213
386
742
405
508
615
95
13
153
150
166
262
519
42
124
632
312
682
775
319
599
251
187
144
154
585
426
575
765
441
500
712
55
741
457
636
382
168
12
159
88
735
475
281
189
71
340
738
789
34
248
305
616
18
232
137
613
345
463
278
799
425
637
418
285
446
203
198
766
149
215
733
755
352
121
211
635
621
81
460
771
86
484
103
711
309
228
412
445
223
528
774
362
751
174
57
410
292
486
361
655
684
784
134
638
611
788
170
44
493
546
722
237
524
396
593
690
349
489
392
339
342
368
728
20
680
506
11
433
448
328
39
94
75
269
745
562
286
195
685
614
640
544
694
610
793
371
147
197
714
450
737
483
731
91
644
186
442
229
89
259
627
135
794
314
648
177
26
90
380
459
792
509
28
200
510
373
732
99
7
293
536
76
298
324
113
263
230
33
628
320
501
577
384
307
130
670
347
597
683
543
417
14
720
769
104
98
50
343
289
270
15
359
355
387
538
453
46
726
249
555
488
397
274
454
721
447
331
639
378
491
586
796
284
444
668
327
724
776
649
29
552
753
97
439
531
142
288
238
277
188
28